In [1]:
import pandas as pd

# Get chemistry data

* Download chemistry data for the NIVA river monitoring stations (elveovervåkingsprogrammet):
    * Locations: Drammenselva, Numedalslågen and Glomma.
    * Get the data from NIVABasen (or similar internal NIVA system) initially. (Note: there is already chemistry data in this folder: aquainfra/data/river/water_chem/raw_vannmiljo, but it has data gaps for some reason. Lets delete this and start with NIVA data for now).
    * Parameters of potential interest: SPM, DOC, TOC, SiO2, TP, TN, NOx-N, NH4-N, SRP, TDP, FDOM or CDOM (if measured?). To discuss with Helene which of these we ultimately want to focus on in AquaInfra
    * Save raw data e.g. here: aquainfra/data/river/water_chem/raw_niva (need to make that last folder)
    * Future extension: download chemistry data from Vannmiljø using the Vannmiljø API
* Quick QC of the data.
* Clean data if necessary and standardise column naming
* Save as csvs in aquainfra/data/river/water_chem/cleaned (need to make that last folder)
* Also save water chemistry site metadata to csv (e.g. station name, station id, lat, lon, any other info we have)